In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

import pandas as pd
import jieba
import re

In [2]:
class MyDataset(Dataset):
    def __init__(self, max_len, debug=True):
        super().__init__()
        df = pd.read_csv('../../datasets/THUCNews/train.csv')
        df = df.dropna()
        if debug:
            df = df.sample(2000).reset_index(drop=True)
        else:
            df = df.sample(50000).reset_index(drop=True)
        # 读取常用停用词
        stopwords = [line.strip() for line in open('../../stopwords/cn_stopwords.txt', 'r', encoding='utf-8').readlines()]
        sentences = []
        for title in df['title']:
            # 去除标点符号
            title = re.sub(r'[^\u4e00-\u9fa5]', '', title)
            # jieba分词
            sentence_seged = jieba.cut(title.strip())
            outstr = ''
            for word in sentence_seged:
                if word != '\t' and word not in stopwords:
                    outstr += word
                    outstr += ' '
            if outstr != '':
                sentences.append(outstr)
        # 获取所有词(token), <pad>用来填充不满足max_len的句子
        token_list = ['<pad>'] + list(set(' '.join(sentences).split()))
        # token和index互转字典
        self.token2idx = {token: i for i, token in enumerate(token_list)}
        self.idx2token = {i: token for i, token in enumerate(token_list)}
        self.vocab_size = len(self.token2idx)

        self.inputs = []
        for sentence in sentences:
            tokens = sentence.split()
            input_ = [self.token2idx[token] for token in tokens]
            if len(input_) < max_len:
                self.inputs.append(input_ + [self.token2idx['<pad>']] * (max_len - len(input_)))
            else:
                self.inputs.append(input_[: max_len])

        self.labels = [[label] for label in df['label'].values.tolist()]
        self.n_class = len(df['label'].unique())

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return torch.LongTensor(self.inputs[idx]), torch.LongTensor(self.labels[idx])


class TextGRU(nn.Module):
    def __init__(self, vocab_size, embed_size, n_class, n_hidden):
        super().__init__()
        # 嵌入层
        self.embedding = nn.Embedding(vocab_size, embed_size)
        # rnn层，深度为1
        self.gru = nn.GRU(embed_size, n_hidden, 1, batch_first=True)
        # 激活函数
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.1)
        # 输出头
        self.fc = nn.Linear(n_hidden, n_class)

    def forward(self, x):  # x: [batch_size * 句子长度]
        x = self.embedding(x)  # [batch_size * 句子长度 * embed_size]
        out, _ = self.gru(x)  # [batch_size * 句子长度 * n_hidden]
        out = self.relu(out)
        out = self.dropout(out)
        logits = self.fc(out[:,-1,:])  # 全连接输出头，[batch_size * n_class]
        return logits

In [4]:
dataset = MyDataset(max_len=10)  # 构造长度为10的句子输入，超过10的句子切掉，不足10的补<pad>
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)
model = TextGRU(vocab_size=dataset.vocab_size, embed_size=128,
                n_class=dataset.n_class, n_hidden=256)
print(model)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()
for epoch in range(50):
    for feature, target in dataloader:
        optimizer.zero_grad()
        logits = model(feature)
        loss = criterion(logits, target.squeeze())
        loss.backward()
        optimizer.step()
    print('epoch:', epoch + 1, ', loss:', loss.item())

TextGRU(
  (embedding): Embedding(8097, 128)
  (gru): GRU(128, 256, batch_first=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.1, inplace=False)
  (fc): Linear(in_features=256, out_features=14, bias=True)
)
epoch: 1 , loss: 1.9135102033615112
epoch: 2 , loss: 1.6494836807250977
epoch: 3 , loss: 1.4462834596633911
epoch: 4 , loss: 0.4588618874549866
epoch: 5 , loss: 0.377926766872406
epoch: 6 , loss: 0.40014171600341797
epoch: 7 , loss: 0.18146666884422302
epoch: 8 , loss: 0.05226273834705353
epoch: 9 , loss: 0.034725628793239594
epoch: 10 , loss: 0.009227114729583263
epoch: 11 , loss: 0.006956214085221291
epoch: 12 , loss: 0.005506675224751234
epoch: 13 , loss: 0.006235456094145775
epoch: 14 , loss: 0.0018697641789913177
epoch: 15 , loss: 0.0010580566013231874
epoch: 16 , loss: 0.0026917557697743177
epoch: 17 , loss: 0.002009385032579303
epoch: 18 , loss: 0.0010581776732578874
epoch: 19 , loss: 0.0007392847328446805
epoch: 20 , loss: 0.001907093683257699
epoch: 21 , loss: 0.0020213059

In [5]:
df = pd.read_csv('../../datasets/THUCNews/train.csv')
predict = model(feature).max(1)[1].tolist()
for i in range(len(feature.tolist())):
    print(' '.join([dataset.idx2token[idx] for idx in feature.tolist()[i]]),
          '---> label:',
          dict(zip(df['label'], df['class']))[predict[i]])

带 了解 愤怒 小鸟 背后 制作 秘辛 <pad> <pad> <pad> ---> label: 游戏
港股 名博 恒指 本周 可能 企稳 <pad> <pad> <pad> <pad> ---> label: 股票
奥斯卡 影帝 尼古拉斯 豪宅 装修图 <pad> <pad> <pad> <pad> <pad> ---> label: 家居
任命 前 高盛 董事长 <pad> <pad> <pad> <pad> <pad> <pad> ---> label: 股票
苹果 要求 法官 撤销 反垄断 诉讼 <pad> <pad> <pad> <pad> ---> label: 科技
华讯 财经 严重 失血 反弹 进行 时 <pad> <pad> <pad> ---> label: 股票
山推股份 业绩 低点 已现 看好 出口 复苏 <pad> <pad> <pad> ---> label: 股票
洛佩斯 垂涎 鲨鱼 骇人 体魄 偶像 交手 担心 裁判 偏哨 ---> label: 体育
削减 国际 广播 节省成本 <pad> <pad> <pad> <pad> <pad> <pad> ---> label: 股票
男子 不满 分手 要求 当街 捅 伤 女友 图 <pad> ---> label: 社会
文明 防 血型 特技 组合 <pad> <pad> <pad> <pad> <pad> ---> label: 游戏
李克强 重庆 考察 强调 进一步 推进 西部 开发 <pad> <pad> ---> label: 时政
调 结构 可能 主导 阶段性 预期 <pad> <pad> <pad> <pad> ---> label: 股票
月 新 发布 手机 产品 平均 厚度 降 毫米 以下 ---> label: 科技
提高 分红 率 有助于 解决 大小 非 问题 <pad> <pad> ---> label: 股票
连阳 沪 指 上升 动能 有所 减缓 <pad> <pad> <pad> ---> label: 股票
